# Work in progress

In [1]:
import pyspark

sc = pyspark.SparkContext('local[*]')


In [2]:
from numpy import array
from math import pow

from pyspark.mllib.clustering import KMeans, KMeansModel

file = "/home/jovyan/Resources/tx.csv"
text = sc.textFile(file)

print(text.count())

1265


In [4]:
from numpy import array
from math import pow

from pyspark.mllib.clustering import KMeans, KMeansModel

file = "/home/jovyan/Resources/tx.csv"
text = sc.textFile(file)

txns = text.map(lambda st: st.split(",")).map(lambda el: (el[0], el[1], float(el[2])))

# get the amounts for clustering training
amnts = txns.map(lambda v: array([v[2]]))

clusterCount = 10
iterationCount = 20
model = KMeans.train(amnts, clusterCount, iterationCount)

print("Cluster centers")
for c, value in enumerate(model.centers):
    print(c, value[0])

# generate the amounts for clustering: +- 5*10^i, i=0..3
vals = sc.parallelize(range(4)).map(lambda i: pow(10, i)).flatMap(lambda x: array([-5*x, 5*x])).map(lambda v: array([v]))

# predict the clusters for the amounts
clsd = vals.map(lambda s: (s, model.predict(s)))

print("Predictions")
for value, c in clsd.collect():
    print(value[0], c)

Cluster centers
0 -819.5775
1 -13.2889871944
2 2209.03
3 825.062
4 -316.894
5 1440.24
6 137.718378378
7 -155.603709677
8 -60.8099625468
9 389.824615385
Predictions
-5.0 1
5.0 1
-50.0 8
50.0 1
-500.0 4
500.0 9
-5000.0 0
5000.0 2
